In [2]:
import decimal
decimal.getcontext().rounding = decimal.ROUND_CEILING
with open('./traffic.conf','r') as f:
    data = f.readlines()

In [4]:
data

['T0 9,7,4,6 0.075,0.025,0.052,0.070\n',
 'T1 5,9,7,4 0.035,0.095,0.125,0.013\n',
 'T2 9,3,10,2 0.123,0.036,0.092,0.027\n',
 'T3 5,2,7,6 0.012,0.098,0.075,0.046\n']

In [36]:
def traffic_gen(data_idx):
    # traffic generation
    link_rate = 1e9
    factor = 336 / 8
    # get packetlength + rate
#     data_idx = 0
    name = data[data_idx].split(' ')[0]
    pkt_length = data[data_idx].split(' ')[1].split(',')
    rate = data[data_idx].split(' ')[2].split('\n')[0].split(',')
    template = """*.{name}.numApps = 4
*.{name}.app[*].typename = "UdpSourceApp"
*.{name}.app[0..3].display-name = "Class A" + string(index)
*.{name}.app[0..3].io.destAddress = "L0"
*.{name}.app[0..3].io.destPort = 1000 + parent.index
    """
    subtemplate="""
# Flow {idx}
*.{name}.app[{idx}].source.packetLength = {length} B - 58B
*.{name}.app[{idx}].source.productionInterval = {interval}ms
*.{name}.app[{idx}].source.initialProductionOffset = {interval}ms
    """
    ret = template.format(name=name)

    for i in range(0,4):
        length = int(int(pkt_length[i])*factor)
        interval = round(length / (link_rate/8*float(rate[i])) * 1000,4)
        interval = decimal.Decimal(interval).quantize(decimal.Decimal("0.0001"))
        ret += subtemplate.format(name=name,idx=i,length=length,interval=interval)
    print(ret)
for i in range(0,4):
    traffic_gen(i)

*.T0.numApps = 4
*.T0.app[*].typename = "UdpSourceApp"
*.T0.app[0..3].display-name = "Class A" + string(index)
*.T0.app[0..3].io.destAddress = "L0"
*.T0.app[0..3].io.destPort = 1000 + parent.index
    
# Flow 0
*.T0.app[0].source.packetLength = 378 B - 58B
*.T0.app[0].source.productionInterval = 0.0403ms
*.T0.app[0].source.initialProductionOffset = 0.0403ms
    
# Flow 1
*.T0.app[1].source.packetLength = 294 B - 58B
*.T0.app[1].source.productionInterval = 0.0941ms
*.T0.app[1].source.initialProductionOffset = 0.0941ms
    
# Flow 2
*.T0.app[2].source.packetLength = 168 B - 58B
*.T0.app[2].source.productionInterval = 0.0258ms
*.T0.app[2].source.initialProductionOffset = 0.0258ms
    
# Flow 3
*.T0.app[3].source.packetLength = 252 B - 58B
*.T0.app[3].source.productionInterval = 0.0288ms
*.T0.app[3].source.initialProductionOffset = 0.0288ms
    
*.T1.numApps = 4
*.T1.app[*].typename = "UdpSourceApp"
*.T1.app[0..3].display-name = "Class A" + string(index)
*.T1.app[0..3].io.destAddress = "L0

In [28]:
rate

['0.075', '0.025', '0.052', '0.070']

In [3]:
def filter_conf_gen(data_idx):
    # egress per-stream filtering
    link_rate = 1e9
    factor = 336 / 8
    # get packetlength + rate
    # data_idx = 0
    name = data[data_idx].split(' ')[0]
    pkt_length = data[data_idx].split(' ')[1].split(',')
    rate = data[data_idx].split(' ')[2].split('\n')[0].split(',')
    template = """
*.{name}.bridging.streamFilter.egress.numStreams = 4
*.{name}.bridging.streamFilter.egress.classifier.mapping = {{"Class A0": 0, "Class A1": 1, "Class A2": 2, "Class A3": 3}}
    """
    subtemplate = """
*.{name}.bridging.streamFilter.egress.meter[{idx}].committedInformationRate = {rate}kbps
*.{name}.bridging.streamFilter.egress.meter[{idx}].committedBurstSize = {length}B
    """
    ret = template.format(name=name)
    for i in range(0,4):
        length = int(int(pkt_length[i])*factor)
        r = link_rate*float(rate[i]) / 1000 / 1000
        ret += subtemplate.format(name=name,idx=i,rate=r,length=length)
    print(ret)
for i in range(0,4):
    filter_conf_gen(i)
    


*.T0.bridging.streamFilter.egress.numStreams = 4
*.T0.bridging.streamFilter.egress.classifier.mapping = {"Class A0": 0, "Class A1": 1, "Class A2": 2, "Class A3": 3}
    
*.T0.bridging.streamFilter.egress.meter[0].committedInformationRate = 75.0kbps
*.T0.bridging.streamFilter.egress.meter[0].committedBurstSize = 378B
    
*.T0.bridging.streamFilter.egress.meter[1].committedInformationRate = 25.0kbps
*.T0.bridging.streamFilter.egress.meter[1].committedBurstSize = 294B
    
*.T0.bridging.streamFilter.egress.meter[2].committedInformationRate = 52.0kbps
*.T0.bridging.streamFilter.egress.meter[2].committedBurstSize = 168B
    
*.T0.bridging.streamFilter.egress.meter[3].committedInformationRate = 70.0kbps
*.T0.bridging.streamFilter.egress.meter[3].committedBurstSize = 252B
    

*.T1.bridging.streamFilter.egress.numStreams = 4
*.T1.bridging.streamFilter.egress.classifier.mapping = {"Class A0": 0, "Class A1": 1, "Class A2": 2, "Class A3": 3}
    
*.T1.bridging.streamFilter.egress.meter[0].com

In [68]:
pkt_lens = []
for data_idx in range(0,4):
    pkt_lens.extend([int(a) for a in data[data_idx].split(' ')[1].split(',')])

In [70]:
sum(pkt_lens) * factor

3990.0